# CycleGAN Practice

In [26]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf


from bayes_opt import BayesianOptimization

from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
#import tensorflow_datasets as tfds
from tensorflow_examples.models.pix2pix import pix2pix

from tqdm.auto import tqdm, trange
import os
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np
from sten import Sten


import random


#tfds.disable_progress_bar()
AUTOTUNE = tf.data.experimental.AUTOTUNE

Grabbing the data

In [27]:
#!pip install  bayesian-optimization
#!pip install -q git+https://github.com/tensorflow/examples.git

In [28]:
SEED=1

In [29]:
tf.random.set_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [30]:
OUTPUT_CHANNELS = 3
# pix2pix it has trained of an ordered pair
# pix2pix pretrained model and it returns a model
#generator_g, discriminato_x collection 1 horse 
generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')
generator_f = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')
# discriminator_x zebras 
discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

In [31]:
#LAMBDA = 10 # learning rate

loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [32]:
def discriminator_loss(real, generated):
    real_loss = loss_obj(tf.ones_like(real), real)
    generated_loss = loss_obj(tf.zeros_like(generated), generated)
    total_disc_loss = real_loss + generated_loss
    return total_disc_loss * 0.5 #cute the discriminator in half so we dont over shot

def generator_loss(generated):
      return loss_obj(tf.ones_like(generated), generated)

In [33]:
def calc_cycle_loss(LAMBDA, real_image, cycled_image):
      loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))
  
      return int(LAMBDA) * loss1

In [34]:
def identity_loss(LAMBDA, real_image, same_image):
        loss = tf.reduce_mean(tf.abs(real_image - same_image))
        return int(LAMBDA) * 0.5 * loss

In [35]:
generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [36]:
# #after every epoch save where we are 
# checkpoint_path = "./checkpoints/train"

# ckpt = tf.train.Checkpoint(generator_g=generator_g,
#                            generator_f=generator_f,
#                            discriminator_x=discriminator_x,
#                            discriminator_y=discriminator_y,
#                            generator_g_optimizer=generator_g_optimizer,
#                            generator_f_optimizer=generator_f_optimizer,
#                            discriminator_x_optimizer=discriminator_x_optimizer,
#                            discriminator_y_optimizer=discriminator_y_optimizer)

# ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# # if a checkpoint exists, restore the latest checkpoint.
# if ckpt_manager.latest_checkpoint:
#     ckpt.restore(ckpt_manager.latest_checkpoint)
#     print ('Latest checkpoint restored!!')

In [37]:
@tf.function
def train_step(LAMBDA,real_x, real_y): # horse and zebra
  # persistent is set to True because the tape is used more than
  # once to calculate the gradients.
  # tap automatically finds the derivative of the value (the value you pass to it) for you 
  with tf.GradientTape(persistent=True) as tape:
    # Generator G translates X -> Y
    # Generator F translates Y -> X.
    # G connected to Y
    # F connected to X
    fake_y = generator_g(real_x, training=True) # real horse to zebra
    
    cycled_x = generator_f(fake_y, training=True) #zebra to horse| cycled_x should be really close to real_x

    fake_x = generator_f(real_y, training=True) #real zebra to horse
    cycled_y = generator_g(fake_x, training=True) #horse to zebra

    # same_x and same_y are used for identity loss.
    same_x = generator_f(real_x, training=True) # horse to horse
    same_y = generator_g(real_y, training=True) # zebra to zebra

    disc_real_x = discriminator_x(real_x, training=True) # take a real horse and predict if its real or not
    disc_real_y = discriminator_y(real_y, training=True) # take a real zebra and predict if its real or not

    disc_fake_x = discriminator_x(fake_x, training=True) # does the fake horse look real or not
    disc_fake_y = discriminator_y(fake_y, training=True) # does the fake zebra look real or not

    # calculate the loss
    # we want to see how far the fake image is to the real one
    gen_g_loss = generator_loss(disc_fake_y) # zebra 
    gen_f_loss = generator_loss(disc_fake_x) # horse
    
    
    # it says real_x should be the same as Cycled_x and same does for Y
    total_cycle_loss = calc_cycle_loss(LAMBDA,real_x, cycled_x) + calc_cycle_loss(LAMBDA,real_y, cycled_y)
       
    # Total generator loss = adversarial loss + cycle loss
    # identity loss(): if generator_g zebra it should output a zebra. It job is to understand what makes a zebra a zebra
    # zebra to horse, input  should generate the same as original input, 
    total_gen_g_loss = gen_g_loss + total_cycle_loss + identity_loss(LAMBDA, real_y, same_y)
    total_gen_f_loss = gen_f_loss + total_cycle_loss + identity_loss(LAMBDA, real_x, same_x)
    
    disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x) # takes in the real and fake horse, and rearrange your weights to make a real horse
    disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y) # this is what a real and fake zebra looks like, figure out how close are the weights
  
  # Calculate the gradients for generator and discriminator
  # calculate the derivate and update the weights to improve 
  generator_g_gradients = tape.gradient(total_gen_g_loss,generator_g.trainable_variables)
  generator_f_gradients = tape.gradient(total_gen_f_loss, generator_f.trainable_variables)
  discriminator_x_gradients = tape.gradient(disc_x_loss,discriminator_x.trainable_variables)
  discriminator_y_gradients = tape.gradient(disc_y_loss, discriminator_y.trainable_variables)
  
 #Apply the gradients to the optimizer
  generator_g_optimizer.apply_gradients(zip(generator_g_gradients,generator_g.trainable_variables))
  generator_f_optimizer.apply_gradients(zip(generator_f_gradients, generator_f.trainable_variables))
  discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients,discriminator_x.trainable_variables))
  discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients,discriminator_y.trainable_variables))



In [38]:
# EPOCHS=100
# for epoch in range(EPOCHS):
#     start = time.time()
#     n = 0
#     for i in range(60):
#         image_x=np.load("/home/gm3g/project/S20-team6-project/encodedArray/%d.npy"%(i))
#         image_y=np.load("/home/gm3g/project/S20-team6-project/decodedArray/%d.npy"%(i))
#         train_step(np.asarray([image_x/255.0], dtype='float32'), np.asarray([image_y/255.0], dtype='float32'))
#         #train_step(image_x, image_y) # x encoded array, y decoded array
#         if n % 10 == 0:
#             print ('.', end='')
#         n+=1

#     clear_output(wait=True)
#   # Using a consistent image (sample_horse) so that the progress of the model
#   # is clearly visible.
#     #generate_images(generator_g, sample_horse)
    
#     if (epoch + 1) % 5 == 0:
#         ckpt_save_path = ckpt_manager.save()
#         print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,ckpt_save_path))
#     print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,time.time()-start))

In [39]:
def generate_images(model, test_input):
    prediction = model(test_input)
    plt.figure(figsize=(12, 12))
    display_list = [test_input[0], prediction[0]]
    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')
    plt.show()

In [40]:
def mse(imageA, imageB):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return -err

In [41]:
def Black_Box(EPOCHS,N,LAMBDA,steps_per_epochs):
   

    for epoch in trange(int(EPOCHS),desc='epochs'):
        start = time.time()
#         n = 0
        m=0
#         for _ in trange(int(steps_per_epochs), desc='steps_per_epochs'):
#             i=np.random.randint(1,11)
#             j= np.random.randint(1,6)
        for i in trange(1,11):
            for j in trange(1,6):

                image_x=np.load("/home/gm3g/project/S20-team6-project/encodedArray/{}_{}.npy".format(j,m))
                image_y=np.load("/home/gm3g/project/S20-team6-project/decodedArray/%d.npy"%(m))
                train_step(LAMBDA,np.asarray([image_x/255.0], dtype='float32'), np.asarray([image_y/255.0], dtype='float32'))
                m+=1
        #train_step(image_x, image_y) # x encoded array, y decoded array
#         m+=1
#     if n % 10 == 0:
#         print ('.', end='')
#     n+=1

        #clear_output(wait=True)
      # Using a consistent image (sample_horse) so that the progress of the model
      # is clearly visible.
        #generate_images(generator_g, sample_horse)

#         if (epoch + 1) % 5 == 0:
#             ckpt_save_path = ckpt_manager.save()
#             print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,ckpt_save_path))
#         print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,time.time()-start))
        
    s_t = Sten(int(N))
    for _ in range(20):
        num = np.random.randint(0,50)
        encoded = np.load(os.getcwd() + "/encodedArray/{0}.npy".format(num))
        decoded = np.load(os.getcwd() + "/decodedArray/{0}.npy".format(num))   
        generate_images(generator_g, np.asarray([encoded/255.0], dtype='float32'))
        
        
    sum = 0.0
    for x in all_encoded_files:
        # Get the number for the hidden file so you can grab it from the hidden 
        hidden_file = x[11:14]
        # Get the hidden image file
        hidden = os.getcwd() + "/data/set2/{0}.jpg".format(hidden_file)
        #Get the difference between the 
        sum += mse(generator_g(x), mpimg.imread(hidden)/255.0)
    avg = sum / len(all_encoded_files)
    
    return avg

    
     

In [42]:
bounds={'EPOCHS':(5,10),
        'N':(1,7),
        'LAMBDA':(8,12),
       'steps_per_epochs':(1,5)
       }


In [43]:
optimizer = BayesianOptimization(
    f=Black_Box,
    pbounds=bounds,
    random_state=1
)
# from bayes_opt.util import load_logs
# load_logs(new_optimizer, logs=["./logs.json"])
# #optimizer.maximize(n_iter=10)

In [44]:
logger = JSONLogger(path="./logs.json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

# Results will be saved in ./logs.json
optimizer.maximize(init_points=2,n_iter=10)

KeyboardInterrupt: 

In [ ]:
# from bayes_opt.util import load_logs


# # new_optimizer = BayesianOptimization(
# #     f=black_box_function,
# #     pbounds={"x": (-2, 2), "y": (-2, 2)},
# #     verbose=2,
# #     random_state=7,
# # )

# # New optimizer is loaded with previously seen points
# load_logs(new_optimizer, logs=["./logs.json"])

In [ ]:
# s_t = Sten(7)
# for _ in range(20):
#     num = np.random.randint(0,50)
#     encoded = np.load(os.getcwd() + "/encodedArray/{0}.npy".format(num))
#     decoded = np.load(os.getcwd() + "/decodedArray/{0}.npy".format(num))   
#     generate_images(generator_g, np.asarray([encoded/255.0], dtype='float32'))

In [ ]:
def generate_images(model, test_input):
    prediction = model(test_input)
    plt.figure(figsize=(12, 12))
    display_list = [test_input[0], prediction[0]]
    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')
    plt.show()